In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,43452
2,Huelva,Confirmados PDIA 14 días,2033
3,Huelva,Tasa PDIA 14 días,"396,1650135432703"
4,Huelva,Confirmados PDIA 7 días,908
5,Huelva,Tasa PDIA 7 dias,"176,9394157881404"
6,Huelva,Total Confirmados,43651
7,Huelva,Curados,37382
8,Huelva,Fallecidos,400


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  43452.0


/tmp/ipykernel_3302/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12120.0


/tmp/ipykernel_3302/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3302/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3302/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3302/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 582 personas en los últimos 7 días 

Un positivo PCR cada 262 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,43452.0,908.0,2033.0,513170.0,176.939416,396.165014,165.0
Huelva-Costa,25636.0,606.0,1308.0,289548.0,209.291724,451.738572,112.0
Condado-Campiña,13569.0,216.0,556.0,156231.0,138.256812,355.883275,35.0
Huelva (capital),12120.0,247.0,548.0,143837.0,171.722158,380.986811,31.0
Ayamonte,1936.0,86.0,168.0,21104.0,407.505686,796.057619,16.0
Lepe,2881.0,70.0,156.0,27880.0,251.076040,559.540890,16.0
Sierra de Huelva-Andévalo Central,3843.0,73.0,151.0,67391.0,108.323070,224.065528,15.0
Isla Cristina,2957.0,76.0,157.0,21393.0,355.256392,733.884916,15.0
Cartaya,1778.0,63.0,132.0,20083.0,313.698153,657.272320,12.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cañaveral de León,26.0,7.0,11.0,396.0,1767.676768,2777.777778,5.0
Linares de la Sierra,9.0,4.0,5.0,262.0,1526.717557,1908.396947,0.0
Alájar,31.0,6.0,12.0,759.0,790.513834,1581.027668,0.0
Ayamonte,1936.0,86.0,168.0,21104.0,407.505686,796.057619,16.0
Rosal de la Frontera,58.0,12.0,13.0,1697.0,707.130230,766.057749,2.0
Isla Cristina,2957.0,76.0,157.0,21393.0,355.256392,733.884916,15.0
Cabezas Rubias,46.0,2.0,5.0,706.0,283.286119,708.215297,NaN
Villablanca,265.0,8.0,19.0,2885.0,277.296360,658.578856,0.0
Cartaya,1778.0,63.0,132.0,20083.0,313.698153,657.272320,12.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cortegana,284.0,1.0,14.0,4602.0,21.729683,304.215558,1.0,0.071429
Manzanilla,136.0,1.0,6.0,2118.0,47.214353,283.286119,0.0,0.166667
Rociana del Condado,650.0,4.0,21.0,7939.0,50.384179,264.516942,0.0,0.190476
Higuera de la Sierra,56.0,1.0,5.0,1291.0,77.459334,387.296669,0.0,0.200000
Paterna del Campo,191.0,3.0,15.0,3457.0,86.780445,433.902227,0.0,0.200000
Cumbres Mayores,129.0,2.0,9.0,1749.0,114.351058,514.579760,1.0,0.222222
Bonares,442.0,6.0,26.0,6060.0,99.009901,429.042904,0.0,0.230769
Trigueros,527.0,11.0,47.0,7862.0,139.913508,597.812262,4.0,0.234043
Villarrasa,216.0,2.0,7.0,2211.0,90.456807,316.598824,0.0,0.285714
